In [20]:
import paho.mqtt.client as mqtt

client_publish = mqtt.Client("Python2")
client_publish.connect(host="test.mosquitto.org", port=1883)
client_publish.publish("IC.embedded/Iceberg", "Please work dear lord!")
client_publish.disconnect()


0

In [2]:
import smbus
import time

bus = smbus.SMBus(1)
i2c_addr_flex = 0x48 # our flex sensor slave address
i2c_addr_compass = 0x0c # our compass sensor slave address

In [47]:
def read_flex (i2c_addr, duration, no_of_reading):
    #below are the 4 different kind of register in ADC1115 module
    status_reg = 0x00
    mode_reg = 0x01
    dataH_reg = 0x02
    dataL_reg = 0x03
    
    #this is to configure the device
    bus.write_word_data(i2c_addr, mode_reg, 0x8384)

    #this is to write to pointer register
    bus.write_byte(i2c_addr, 0)

    while(no_of_reading):
        val_swapped = bus.read_word_data(i2c_addr, 0x00)
        val = (val_swapped & 0xFF) <<8 | (val_swapped >>8)
        print ("flex reading: ", val)
        time.sleep(duration/no_of_reading)
        no_of_reading = no_of_reading - 1

In [27]:
def read_compass(i2c_addr, duration, no_of_reading):
    
    while(no_of_reading):
        # Select write register command, 0x60(96)
        # AH = 0x00, AL = 0x5C, GAIN_SEL = 5, Address register (0x00 << 2)
        config = [0x00, 0x5C, 0x00]
        bus.write_i2c_block_data(i2c_addr, 0x60, config)

        # Read data back, 1 byte, (ACK byte)
        # Status byte
        data = bus.read_byte(i2c_addr)

        # AH = 0x02, AL = 0xB4, RES for magnetic measurement = 0, Address register (0x02 << 2)
        config = [0x02, 0xB4, 0x08]
        bus.write_i2c_block_data(i2c_addr, 0x60, config)

        # Read data back, 1 bytem (ACK byte)
        # Status byte
        data = bus.read_byte(i2c_addr)

        # Start single meaurement mode, X, Y, Z-Axis enabled
        bus.write_byte(i2c_addr, 0x3E)

        # Read data back, 1 byte
        # Status byte
        data = bus.read_byte(0x0C)

        time.sleep(0.5)

        # Read data back from 0x4E(78), 7 bytes
        # Status, xMag msb, xMag lsb, yMag msb, yMag lsb, zMag msb, zMag lsb
        data = bus.read_i2c_block_data(0x0C, 0x4E, 7)
#         print(data)

        xMag = data[1] * 256 + data[2]
        if xMag > 32767 :
            xMag -= 65536

        yMag = data[3] * 256 + data[4]
        if yMag > 32767 :
            yMag -= 65536

        zMag = data[5] * 256 + data[6]
        if zMag > 32767 :
            zMag -= 65536

        # Output data to screen
        print("Magnetice Field (x, y, z) = ", xMag, yMag, zMag)
        time.sleep(duration/no_of_reading)
        no_of_reading = no_of_reading - 1

        
        #if magnetic field change too much then nearby must have ferous object
        
    

In [53]:
read_flex(i2c_addr_flex, 10, 20)
# read_compass(i2c_addr_compass, 2, 2)

flex reading:  8138
flex reading:  8784
flex reading:  8752
flex reading:  9212
flex reading:  10443
flex reading:  11707
flex reading:  12923
flex reading:  9989
flex reading:  8387
flex reading:  8932
flex reading:  11904
flex reading:  12919
flex reading:  13470
flex reading:  9688
flex reading:  8307
flex reading:  8615
flex reading:  13247
flex reading:  9804
flex reading:  9895
flex reading:  9085


In [52]:
read_compass(i2c_addr_compass, 5, 5)

Magnetice Field (x, y, z) =  -2 -107 22
Magnetice Field (x, y, z) =  -3 -108 19
Magnetice Field (x, y, z) =  -2 -104 21
Magnetice Field (x, y, z) =  3 -104 14


KeyboardInterrupt: 

In [40]:
import time
import paho.mqtt.client as mqtt
print("Done")
client = mqtt.Client("")
print("Here1")
client.connect("test.mosquitto.org", port=1883)
print("Here2")
client.publish("IC.embedded/Faraday", "HEY MAN")
print("Here3")
print("done")

Done
Here1
Here2
Here3
done
